<img src="images/logo.png" width="200">

# KogSys-KI-B - Assignment 2

### Adversarial Search, Constraint Satisfaction Problems

_Abgabefrist: **15.06.2025**_

---



#### Submission Information

Upload your solution via the VC course. Please upload **one Zip** archive per group. This must contain:

- Your solution as **Notebook** (a `.inpynb` file)
- A folder named **images** with all your images, if you used any (keep the image sizes relatively small)

Your Zip file should be named as follows:

```
assignment_<assignment number>_solution_<group number>.zip
```

In this assignment, you can achieve a total of **30** points. From these points, **3 bonus points** for the exam will be calculated as follows:

| **Points in Assignment** | **Bonus Points for Exam** |
| :----------------------: | :-----------------------: |
|            30            |             3             |
|            25            |            2.5            |
|            20            |             2             |
|            15            |            1.5            |
|            10            |             1             |
|            5             |            0.5            |

<div class='alert alert-block alert-danger'>

##### **Important Notes**

1. **This assignment is graded. You can earn bonus points for the exam.**
2. **If it is evident that an assignment was copied from another source and no independent work was done, no bonus points will be awarded. Please formulate all answers in your own words!**
3. **If LLMs (such as ChatGPT or Copilot) were used to create your submission, please indicate this in accordance with common scientific practices. See also the [AI Policy in the VC Course](https://vc.uni-bamberg.de/mod/page/view.php?id=1980835)**

### Setup

To setup your assignment, you need to install the required dependencies, specified in `requirements.txt`. You can do this by executing the following code cell.

In [ ]:
# Installiert die benötigten Pakete mit dem akutell ausgewählten Python-Interpreter
%pip install -U -r requirements.txt

# Aufgabe 2 - Constraint Satisfaction Problems (CSP) with the example "Hashiwokakero"

_For totally <mark>15</mark> points_

This task implements a **Constraint Satisfaction Problem (CSP) solver** for the [Hashiwokakero puzzle game](https://en.wikipedia.org/wiki/Hashiwokakero). Your task is to complete the CSP solver implementation.

You can try playing the game here: https://www.hashi.info/.

## CSP formulation:
* **Variables (X)**: Potential bridges between neighboring islands
* **Domains (D)**: Number of bridges (0, 1, or 2) between pairs of islands
* **Constraints (C)**:
  - Each island must have exactly the number of bridges indicated by its value
  - Bridges cannot cross each other
  - Bridges can only run horizontally or vertically
  - All islands must be connected to form a connected graph


#### Library imports

The following library imports may be used throughout this task. Do not use any third-party libraries.

- `collections.deque` and `collections.defaultdict`: These are used for implementing the data structures needed for connectivity checking.
- `time`: This is used to implement a timeout for the CSP solver.
- `typing.List`, `typing.Tuple`, `typing.Dict`, `typing.Optional` und `typing.Iterable`: These are used for type annotations in method specifications.
- `pprint`: Used for printing data structures in a more readable format.


**_Nothing needs to be changed in the next code cell._**

In [ ]:
from collections import deque, defaultdict
import time
from typing import List, Tuple, Dict, Optional, Iterable
from pprint import pprint

## Given Helper Functions

**_The helper functions do not need to be changed. However, try to understand them so you can use them when necessary._**


### 0. Data Structures and Game Board

* Given data structures of the game board:
  * The `Island` class contains the position and it's required number of bridges. 
  * The `HashiwokakeroBoard` is the implementation of the game board. It contains logic to:
    * get an island from specific coordinates (`get_island`),
    * add bridges (`add_bridge`),
    * get all possible connections between islands (`find_potential_connections`).
  * The `HashiwokakeroGameEngine` implements the game's display and interactions using pygame:
    * `solve` in the constructor is a function intended to solve the CSP (Constraint Satisfaction Problem).
    * `run` starts the game and displays the game board.\
      In this mode, several shortcuts are available, described under [Game Launch](#game-launch).
    * `board` is the current game board.
    * `load_puzzle` loads a puzzle by name (`puzzle1`, `puzzle2`, or `puzzle3`) or with a list of islands in the format `list[tuple[row: int, col: int, value: int]]`.
  * Data structures for the CSP solver:
    * **`type Var_PossibleBridge`**: Represents a possible bridge between two islands. (Variable `x` of the CSP)
    * **`domain`**: All possible values for a variable (number of bridges between two islands: 0, 1, or 2).
    * **`type Assignment`**: Represents an assignment of variables to values. It is a dictionary that assigns the possible bridges (`Var_PossibleBridge`) to values from the `domain` or `None` if the variable is not yet assigned.\
      Helper functions for working with `Assignment`:
      * `get_bridge_count`: Determines the number of bridges for an `Island` in an `Assignment`.
      * `get_unassigned_vars`: Returns a list of all unassigned variables in an `Assignment`. Optionally for a specific `Island`.

Example:
```python
#                row, col, value
island1 = Island(0,   0,   2    )
island2 = Island(0,   1,   1    ) 
island3 = Island(1,   0,   1    )
board = HashiwokakeroBoard(islands=[island1, island2, island3])

assignment: Assignment = {
    (island1, island2): 1, # 1 bridge from island1 to island2
    (island1, island3): None # no bridge count assigned yet
}
```

In [ ]:
from hashiwokakero import Island, HashiwokakeroBoard, HashiwokakeroGameEngine

type Var_PossibleBridge = Tuple[Island, Island]

domain = (0, 1, 2)

type Assignment = Dict[Var_PossibleBridge, Optional[int]]

### General Helper functions ###

def get_bridge_count(island: Island, assignment: Assignment) -> int:
    """
    Gibt die Anzahl der Brücken zurück, die aktuell für eine Insel zugewiesen sind.
    """
    return sum(val for var, val in assignment.items() if island in var and val is not None)

def get_unassigned_vars(assignment: Assignment, island: Optional[Island] = None) -> List[Var_PossibleBridge]:
    """
    Gibt eine Liste aller Variablen zurück, die unzugewiesen sind (und optional zu der angegebenen Insel gehören).

    Args:
        assignment (Assignment): Aktuelle Zuweisung der Variablen.
        island (Optional[Island], optional): Wenn gegeben alle unzugewiesenen Variablen mit der gegebenen Insel. Defaults to None.
    """
    return [var for var, val in assignment.items() if island is None or (island in var and val is None)]

### 1. Checking Island Satisfiability

Calculates the current number of bridges for an island in an assignment and checks if the island's target is still reachable.


In [ ]:

def is_island_satisfiable(island: Island, assignment: Assignment) -> bool:

    current_bridges = get_bridge_count(island, assignment)

    # Kann geforderte Brückenanzahl nicht überschreiten
    if current_bridges > island.value:
        return False

    remaining_needed = island.value - current_bridges

    # Bereits erfüllt
    if remaining_needed == 0:
        return True

    unassigned_vars = get_unassigned_vars(assignment, island)

    # Prüfen ob Ziel noch erreichbar ist
    max_possible_additional = len(unassigned_vars) * max(domain)

    return max_possible_additional >= remaining_needed


### 2. Check if a new bridge can be created

Checks if a new bridge can be created between two islands without crossing an existing bridge.

Uses a helper function `do_bridges_cross` to check if two bridges cross each other.



In [ ]:
def do_bridges_cross(bridge1: Var_PossibleBridge, bridge2: Var_PossibleBridge) -> bool:

    (island1_1, island1_2) = bridge1
    (island2_1, island2_2) = bridge2

    # Calculate if the bridges are horizontal or vertical
    bridge1_horizontal = (island1_1.row == island1_2.row)
    bridge2_horizontal = (island2_1.row == island2_2.row)

    if bridge1_horizontal == bridge2_horizontal:
        return False  # Both bridges are either horizontal or vertical, so they cannot cross

    # Depending on whether the first bridge is horizontal or vertical, set the coordinates
    # and determine the boundaries of the other bridge
    if bridge1_horizontal:
        h_row = island1_1.row
        h_col_min, h_col_max = min(island1_1.col, island1_2.col), max(island1_1.col, island1_2.col)
        v_col = island2_1.col
        v_row_min, v_row_max = min(island2_1.row, island2_2.row), max(island2_1.row, island2_2.row)
    else:
        h_row = island2_1.row
        h_col_min, h_col_max = min(island2_1.col, island2_2.col), max(island2_1.col, island2_2.col)
        v_col = island1_1.col
        v_row_min, v_row_max = min(island1_1.row, island1_2.row), max(island1_1.row, island1_2.row)

    # Check if they cross
    return (h_col_min < v_col < h_col_max) and (v_row_min < h_row < v_row_max)

def no_bridge_crossing(assignment: Assignment, var: Var_PossibleBridge) -> bool:

    active_bridges = [v for v, val in assignment.items()
                     if val is not None and val > 0 and v != var]

    return all(not do_bridges_cross(var, other_var) for other_var in active_bridges)

### 3. Sicherheitsprüfung für ein Assignment

Checks the assignment for a new bridge and ensures that the connected islands remain satisfiable and that the assignment does not create crossing bridges.

In [ ]:
def is_assignment_consistent(assignment: Assignment, var: Var_PossibleBridge, value: int) -> bool:

    # Create a temporary assignment to check consistency
    temp_assignment = assignment.copy()
    temp_assignment[var] = value

    # Check if both islands are still satisfiable with the new assignment
    return (is_island_satisfiable(var[0], temp_assignment) and
            is_island_satisfiable(var[1], temp_assignment) and
            # If at least one bridge is set, verify that it does not create a crossing
            (value == 0 or no_bridge_crossing(temp_assignment, var)))


### 4. Find Valid Domain Values


Tests all possible domain values (0, 1, 2) for a bridge and filters out invalid options. It may return an empty list if no valid values are found. The return value is sorted from high to low.

In [ ]:
def get_valid_domain_values(assignment: Assignment, var: Var_PossibleBridge) -> Iterable[int]:

    # Werte in Reihenfolge der Präferenz testen (höhere Werte zuerst)
    return filter(lambda v: is_assignment_consistent(assignment, var, v), reversed(domain))

### 5. Check Island Connectivity

Uses breadth-first search (BFS) to check if all islands are reachable via bridges. If `islands=None` (default), only islands that already have bridges are checked.


In [ ]:
def check_island_connectivity(assignment: Assignment, islands: Optional[Iterable[Island]] = None) -> bool:
    # Adjazenzgraph erstellen
    graph = defaultdict(list)
    if islands:
        relevant_islands = set(islands)
    else:
        relevant_islands = set()
    
    for (id1, id2), bridges in assignment.items():
        if bridges is not None and bridges > 0:
            if islands is None:
                relevant_islands.add(id1)
                relevant_islands.add(id2)
            graph[id1].append(id2)
            graph[id2].append(id1)

    # BFS zur Konnektivitätsprüfung
    start_island = next(iter(relevant_islands), None)
    visited = set([start_island])
    frontier = deque([start_island])

    while frontier and len(visited) != len(relevant_islands):
        current = frontier.popleft()
        for neighbor in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                frontier.append(neighbor)

    return len(visited) == len(relevant_islands)


### 6. Validate Solution

Checks if all islands have the correct number of bridges and if all islands are connected.

In [ ]:
def validate_complete_solution(board: HashiwokakeroBoard, assignment: Assignment) -> bool:

    # Prüfen dass alle Inseln exakte Brückenanzahl haben
    for island in board.islands:
        current_bridges = sum(val for var, val in assignment.items() if island in var and val is not None)
        if current_bridges != island.value:
            return False

    # Konnektivität für alle Inseln prüfen
    return check_island_connectivity(assignment, board.islands)


### 7. Apply Solution to Game

Resets the game board and creates bridges based on the given assignment.

In [ ]:
def apply_assignment_to_board(board: HashiwokakeroBoard, assignment: Assignment) -> None:

    board.bridges = []
    
    for (island1, island2), bridges in assignment.items():
        if bridges is not None and bridges > 0:
            board.add_bridge(island1, island2, bridges)


## Implementation


### **(02.2.1)**: Variable-Sorting

*For <mark>3</mark> points*

Implement the function `order_variables_by_constraint`, which sorts the possible variables. Use a combination of the following heuristics for this:

* **MRV (Minimum Remaining Values) heuristic**: Variables with **fewer valid values should be processed first**.
* **Degree heuristic**: The most constrained islands should be considered first.

Of course, you may also use additional helper functions to implement the sorting.


In [ ]:
# Implementation Task 02.2.1

def order_variables_by_constraint(assignment: Assignment, variables: List[Var_PossibleBridge]) -> List[Var_PossibleBridge]:
    """_summary_

    Args:
        assignment (Assignment): current assignment of the variables
        variables (List[Var_PossibleBridge]): Unassigned variables to order	

    Returns:
        List[Var_PossibleBridge]: A list of variables ordered by the MRV and degree heuristic.
    """
    # TODO: Implement
    return variables 



#### Test for task 02.2.1

In [ ]:
def test_order_variables_by_constraint():

    island1 = Island(0, 0, 1)  # Least constrained
    island2 = Island(0, 1, 4)  # Most constrained
    island3 = Island(0, 2, 1)  # Least constrained
    island4 = Island(1, 1, 2)  # Average constrained
    assignment: Assignment = {
        (island1, island2): None,
        (island2, island3): None,
        (island2, island4): None
    }
    variables = list(assignment.keys())


    # Execute the function to test
    ordered = order_variables_by_constraint(assignment, variables)

    print(f"Original variables:")
    pprint(variables)
    print(f"Ordered variables:")
    pprint(ordered)
    print("✓ Test completed - check if most constrained variables are first")
    print()

test_order_variables_by_constraint()

### **(02.2.2)** Backtracking Search

*For <mark>5</mark> points*

Implement the function `backtrack_search`, which implements the backtracking algorithm for the CSP problem.

<details>
<summary>Hints</summary>

* **Sort the unassigned variables** and select the first variable
* Try **each valid value** in order and start a recursive search
* **Copy the `assignment`** before making any changes

</details>


In [ ]:
def backtrack_search(board: HashiwokakeroBoard, assignment: Assignment,
                    timeout_time: float) -> Tuple[bool, Assignment]:

    if time.time() > timeout_time:
        return False, assignment

    # TODO: Implement Backtracking Search

    return False, assignment  


#### Test for 02.2.2

In [ ]:
def test_backtrack_search():
        
    island1 = Island(0, 0, 2)
    island2 = Island(0, 1, 1) 
    island3 = Island(1, 0, 1)
    board = HashiwokakeroBoard(islands=[island1, island2, island3])
    
    assignment: Assignment = {
        (island1, island2): None,
        (island1, island3): None
    }
    timeout_time = time.time() + 10

    success, final_assignment = backtrack_search(board, assignment, timeout_time)
    print(f"Search successful: {success}")
    print(f"Final assignment:")
    pprint(final_assignment)
    print("✓ Test completed - check if backtracking logic works")

test_backtrack_search()

### **(02.2.3)** Main Solving Function

*For <mark>3</mark> points*

Implement the function `solve_hashiwokakero`, which orchestrates the entire CSP solver. This function should call all previously implemented functions and apply the solution to the game board if one is found.

<details>
<summary>Hints</summary>

* **Create an initial assignment where all variables are unassigned**, using the helper function `board.find_potential_connections()`
* **Set a timeout** after which the solver should stop to avoid an endless search (e.g., 60 seconds)
* **Call the function `backtrack_search`**, passing the initial assignment and the maximum runtime
* **If a solution is found, apply it to the game board** using the function `apply_assignment_to_board`

</details>


In [ ]:
def solve_hashiwokakero(board: HashiwokakeroBoard) -> bool:

    print("Starting CSP solver...")
    # TODO: Implement the CSP orchestration and apply the solution to the board if one is found
    return False


#### Test for task 02.2.3

The following game board is used for the test call:


![](https://upload.wikimedia.org/wikipedia/commons/d/d9/Ejemplo_hashiwokakero_5x5.PNG)

In [ ]:
def test_solve_hashiwokakero(verbose=True):
    """Test der Hauptlösungsfunktion"""
    if verbose:
        print("=== Test: Main Solver Function ===")

    # Simple setup without pygame

    board = HashiwokakeroBoard.from_tuple_definition(
        [
            (0, 0, 1),
            (0, 1, 2),
            (0, 2, 1),
            (1, 0, 3),
            (1, 1, 8),
            (1, 2, 5),
            (2, 0, 2),
            (2, 1, 6),
            (2, 2, 4)
        ],
        grid_size=3
    )

    success = solve_hashiwokakero(board)

    print(f"Solver completed: {success}")
    print(f"Final bridges:")
    pprint(board.bridges)
    print("✓ Test completed - check if main solver coordinates all functions")
    print()


test_solve_hashiwokakero()

### Game Launch

Now that you have completed the implementation of the CSP solver, you can launch the game and solve the puzzles automatically!

#### Controls:

* Press **'S'** to solve the current puzzle
* Press **'R'** to reset the puzzle
* Press **'1'**, **'2'**, **'3'** to load different puzzles


In [ ]:
def main():
    """
    Vereinfachte Hauptfunktion für Spielausführung
    """
    # Spiel erstellen
    game = HashiwokakeroGameEngine(solve=solve_hashiwokakero)

    print("=== Hashiwokakero CSP Solver ===")
    print("Controls:")
    print("- Press 'S' to solve current puzzle")
    print("- Press 'R' to reset puzzle")
    print("- Press '1' to load puzzle 1")
    print("- Press '2' to load puzzle 2")
    print("- Press '3' to load puzzle 3")
    print("- Press ESC or close window to quit")
    print()
    
    game.run()


main()

### **(02.2.4)** Weaknesses of the Implementation

*For <mark>4</mark> points*

In this entire exercise, we implemented a basic CSP solver for the Hashiwokakero puzzle. However, the implementation is not necessarily optimal and can be inefficient in some cases. You’ll likely notice this when trying to solve `puzzle3`.

Name **two** possible weaknesses of the implementation and propose solutions to address them.


> Enter your answer here

> 